<a href='https://colab.research.google.com/https://github.com/FaezeMqFr/Variational_Autoencoder/edit/main/VAE_MNIST.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import gzip
import urllib

In [ ]:
# Download the MNIST dataset
url = 'http://yann.lecun.com/exdb/mnist/'
train_images = 'train-images-idx3-ubyte.gz'
train_labels = 'train-labels-idx1-ubyte.gz'
test_images = 't10k-images-idx3-ubyte.gz'
test_labels = 't10k-labels-idx1-ubyte.gz'

In [ ]:
# Helper function to load the dataset
def load_mnist(images_path: str, labels_path: str):
    with gzip.open(urllib.request.urlopen(url + images_path)) as f:
        images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)
    with gzip.open(urllib.request.urlopen(url + labels_path)) as f:
        labels = np.frombuffer(f.read(), np.uint8, offset=8)
    return images, labels

# Load data
train_images, train_labels = load_mnist(train_images, train_labels)
test_images, test_labels = load_mnist(test_images, test_labels)

# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Split train data into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

In [ ]:
# Define the Variational Autoencoder (VAE) model
class VAE(models.Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dense(latent_dim * 2),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(512, activation='relu'),
            layers.Dense(784, activation='sigmoid'),
            layers.Reshape((28, 28))
        ])

    def encode(self, x):
        mean, log_var = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, log_var

    def reparameterize(self, mean, log_var):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(log_var * 0.5) + mean

    def decode(self, z):
        return self.decoder(z)

In [ ]:
# Instantiate the VAE model
latent_dim = 2
vae = VAE(latent_dim)

# Define the VAE loss function
def vae_loss(x, recon_x, mean, log_var):
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=recon_x, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2])
    kl_div = -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean) - tf.exp(log_var), axis=1)
    return -tf.reduce_mean(logpx_z + kl_div)

In [ ]:
# Compile and train the VAE model
vae.compile(optimizer='adam', loss=vae_loss)
vae.fit(train_images, epochs=50, batch_size=64, validation_data=(val_images, val_images))

In [ ]:
# Generate and display images from the VAE
def plot_images(images):
    fig, axes = plt.subplots(1, 10, figsize=(10, 10))
    for img, ax in zip(images, axes):
        ax.set_axis_off()
        ax.imshow(img.reshape(28, 28), cmap='gray')
plot_images(vae.decode(tf.random.normal(shape=(10, latent_dim))))